In [96]:
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString 
import csv  
import pandas as pd

In [97]:
#Variables
subHeading, subHeadingList  =([] for i in range(2))
intro = ''
chapterName = ''

#final answer list
subHeadingIntro, subHeadings, allEaxmples, allListItems, allImages, allTableValues =([] for i in range(6))

In [98]:
#convert list into string
def listIntoString(lists):  
    str1 =''
    for ele in lists:         
        str1 += ele  
    return str1

In [99]:
# here all p tags are passed which includes text all other headings also
# so here exatracting only needed text for 
def filterSubHeadingIntro(subHeadingIntro):
    removeStr = len(SubHeadingIntro)
    i = -1
    subIntro = []
    lastItem = ""
    while removeStr > 0:
        if(len(subHeadingIntro[i]) !=0):
            if(len(subIntro[i]) ==0):
                subIntro.append(subHeadingIntro[i])
                lastItem = subHeadingIntro[i]
            else:
                    subIntro.append(subHeadingIntro[i].replace(lastItem, ""))
                    lastItem = subHeadingIntro[i]
                    i -= 1
                    removeStr -= 1
                    subIntro.reverse()
                    return subIntro

In [100]:
def findHeading(soup):
    #print("\n==========PAGE HEADING==============")
    heading = soup.find_all('span', class_='color_h1')
    if(heading != []):
        heading = heading[0].get_text()
    else:
            heading = ""
            
            
            #print("\n==========INTRODUCTION==============")
            global intro
            intro = soup.find_all('p',class_='intro')
            
            if(intro !=[]):
                intro = intro[0].get_text()
            else:
                    intro = ""
                    
    return heading

In [101]:
def findSubHeading(soup):
    global subHeadings
    try:
        #print("\n==========ALL SUB HEADINGS==============")
        sub_heading = soup.find('hr')
        sub_heading = sub_heading.find_next_siblings("h2")
        for h in sub_heading:
            if(h.string.find('Exercises') !=-1):
                break
            elif(h.string == 'Chapter Summary'):
                break
            elif(h.string == 'HTML List Tags'):
                break
            else:
                subHeadings.append(h.get_text())
                
    except:
        subHeadings.append('')
   

In [102]:
def findIntroSubheading(soup):
    global subHeadingIntro
    #print("\n==========INTRO FOR ALL SUB HEADINGS==============")
    try:
        sub_heading_intro = soup.find('div', id='main')
        length = len(sub_heading_intro.find_all('hr'))
        i=0
        while i < length:
            sub = sub_heading_intro.find_all('hr')[i]
            sub = sub.find_next_siblings("p")
            for k in sub:
                if(k.get_text() == intro):
                    continue
                if(k.get_text().__contains__("W3Schools' tag reference contains")):
                    continue
                else:
                    subHeadingList.append(k.get_text())
                    
                    str1 = listIntoString(subHeadingList)
                    subHeadingList.clear()  
                if(str1 not in subHeadingIntro):
                    subHeadingIntro.append(str1)               
                i += 1
                
    except:
        subHeadingIntro.append('')               
      # sending data for filtering
        subHeadingIntro = filterSubHeadingIntro(subHeadingIntro)   

In [103]:
def findExample(soup):
    global allEaxmples
    dummyExamples = []
    text = ''
    
    #print("\n==========ALL EXAMPLES FOR SUB HEADINGS==============")
    try:
        example = soup.select("div.w3-example div.notranslate")
        for e in example:
            example = False
            dummyExamples.append(e.get_text())
            par = e.parent
            
            #under one heading can have multiple examples
            for sibling in par.next_siblings:
                print(sibling)
                for s in sibling:
                    if(str(s) == '<h3>Example</h3>'):
                        example = True
                        if(example == False):
                            if(str(sibling) == '<hr/>'):
                                dummyExamples.append("|||") 
                                break
                           
       
     # arranging all examples
        for e in dummyExamples: 
            if(e == '|||'):
                allEaxmples.append(text)
                text = ''
            else:
                if(len(text) == 0):
                       text += e
                else:
                    text += '&&&&&&'+e
                    
    except:
        allEaxmples.append("")

In [104]:
#find listItems on webpage
def findListItems(soup):
    global allListItems
    heading = ""
    items = ""
    finalItem = ""
    
    try:
        unorderedList = soup.find_all('ul')
        for k in unorderedList:
            heading += k.find_previous_sibling('h2').get_text() + "###"    #listHeading
            items += "&&&"+k.get_text().replace("\n", "|||")               #listItems
            finalItem = heading + items
            allListItems.append(finalItem)       
    except:
        allListItems.append("")
        #print(allListItems)

In [105]:
#find images on webpage
def findImageUrl(soup):
    global allImages
    
    try:
        content = soup.find("div", id="main")
        images = content.find_all('img')
        for img in images:
            allImages.append(img['src'])
    except:
        allImages.append("")

In [106]:
#find all table values
def findTableValues(soup):
    global allTableValues
    tableText = ""
    
    try:
        content = soup.find("div", id="main")
        tables = content.find_all('table')  
        for table in tables:
            tableRow = table.find_all("tr")
            
            for tr in tableRow:
                if(tr.find_all("th")):
                    tableHeading = tr.find_all("th")
                    for th in tableHeading:
                        tableText += th.get_text() + "|||"
                        
                elif(tr.find_all("td")):
                            tableData = tr.find_all("td")
                            for td in tableData:
                                tableText += td.get_text() + "|||"          
                                tableText += "&&&"
                                
                                allTableValues.append(tableText)
                                tableText = ""
                            
                            
    except:
         allTableValues.append("")
    #print(allTableValues)                                
                                    

In [107]:
def writeInCSV(chapterName, heading, link, intro, subHeadings, subHeadingIntro, allEaxmples, allListItems, allTableValues, allImages):   

  
  for k in range(len(subHeadings)):
    if(k >= len(subHeadingIntro)):
        subHeadingIntro.append("")
    if(k >= len(allEaxmples)):
        allEaxmples.append("")
 

  with open('web.csv', 'a', encoding='UTF8') as f:
    writer = csv.writer(f)
    counter = 0 
    heading = chapterName +'||'+ heading.strip() 

    data = heading, link, intro, subHeadings, subHeadingIntro, allEaxmples, allListItems, allTableValues, allImages
    writer.writerow(data)

In [108]:
header = ['Heading', 'Link', 'Intro', 'Subheading', 'Definitions', 'Example', 'List_item', 'table_item', 'Image_url',]
with open('web.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)


In [109]:
webLinks = pd.read_csv('link1.txt',sep='\n',header=None)[0].tolist()
webLinks = webLinks[:3]
num = 1
for link in webLinks:
    #print(link)
    chapterName = link.split("/")[3]
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup)
    heading = findHeading(soup)
    findSubHeading(soup)  
    findIntroSubheading(soup)
    findExample(soup)
    findListItems(soup)
    findImageUrl(soup)
    findTableValues(soup)
    
    writeInCSV(chapterName, heading, link, intro, subHeadings, subHeadingIntro, allEaxmples, allListItems, allTableValues, allImages)  
    subHeadingIntro.clear()
    subHeadings.clear()
    allEaxmples.clear()  
    allListItems.clear()
    allImages.clear()
    allTableValues.clear()
    #print(num, 'RECORD DONE!')
    num += 1
    
    #print(chapterName)
    #print(heading)
    #print(link)
    #print(intro)
    #print(subHeadings)
    #print(subHeadingIntro)
    #print(allEaxmples)
    #print(allListItems)



<p><strong>Click on the "Try it Yourself" button to see how it works.</strong></p>


<hr/>


<h2>HTML Examples</h2>


<p>In this HTML tutorial, you will find more than 200 examples. With our online 
"Try it Yourself" editor, you can edit and test each example yourself!</p>


<p><a class="w3-button ws-grey w3-round" href="html_examples.asp">Go to HTML Examples!</a></p>


<hr/>


<div id="midcontentadcontainer" style="overflow:auto;text-align:center">
<!-- MidContent -->
<!-- <p class="adtext">Advertisement</p> -->
<div id="div-gpt-ad-1493883843099-0" style="display: inline-block">
<script>
    uic_r_g();
    </script>
</div>
</div>


<hr/>


<h2>HTML Exercises</h2>


<p>This HTML tutorial also contains nearly 100 HTML exercises.</p>


<form action="exercise.asp?filename=exercise_html_attributes1" autocomplete="off" id="w3-exerciseform" method="post" target="_blank">
<h2>Test Yourself With Exercises</h2>
<div class="exercisewindow">
<h2>Exercise:</h2>
<p>Add a "tooltip" to the paragrap

In [110]:
df = pd.read_csv("web.csv")  
df.head(10)

,Heading,Link,Intro,Subheading,Definitions,Example,List_item,table_item,Image_url
0,html||Tutorial,https://www.w3schools.com/html/default.asp,NaN,"['Easy Learning with HTML ""Try it Yourself""', ...","['With our ""Try it Yourself"" editor, you can e...","['', '']",[],[],[]
1,html||Introduction,https://www.w3schools.com/html/html_intro.asp,NaN,"['What is HTML?', 'A Simple HTML Document', 'W...",['HTML is the standard markup language for cre...,"['', '', '', '', '', '']",['What is HTML?###&&&|||HTML stands for Hyper ...,['Start tag|||Element content|||End tag|||<h1>...,['img_chrome.png']
2,html||Editors,https://www.w3schools.com/html/html_editors.asp,NaN,"['Learn HTML Using Notepad or TextEdit', 'Step...",['A simple text editor is all you need to lear...,"['', '', '', '', '', '', '', '']",[],[],"['img_notepad.png', 'img_saveas.png', 'img_chr..."


In [111]:
file1 = open("link1.txt","a")
URL = "https://www.w3schools.com/html/"
r = requests.get(URL)  
soup = BeautifulSoup(r.content, 'html.parser')
divs  = soup.find("div", id="leftmenuinnerinner")
soup.select("div.w3-example div.notranslate")
for a in divs.find_all('a', href=True, target="_top"):
    links = URL + a['href']
    file1.write(links)
    file1.write('\n')
    #print(links)
    #file1.close()